In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.cluster import KMeans
%matplotlib inline
plt.style.use('ggplot')
import seaborn as sns

In [2]:
data_high_freq=pd.read_csv("id_with_1ormore_freq.csv",on_bad_lines="skip")
data_low_freq=pd.read_csv("id_with_0_freq.csv",on_bad_lines="skip")

In [3]:
data_high_freq.drop(['REGION','RG','MSISDN'],axis=1)
data_low_freq.drop(['REGION','RG','MSISDN'],axis=1)
data_high_freq=data_high_freq[data_high_freq['RECHARGE_AMOUNT']!=0].reset_index()
data_high_freq['RATIO']=data_high_freq['TOTAL_DSTR']/data_high_freq['RECHARGE_AMOUNT']
for i in range(len(data_high_freq)):
    if data_high_freq['RATIO'][i]<0.7:
        data_high_freq['RATIO'][i]= 1
    else:
        data_high_freq['RATIO'][i]= 2
data_low_freq['RATIO']=0

C:\Users\zhuhe\AppData\Local\Temp\ipykernel_33196\3774832301.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_high_freq['RATIO'][i]= 2
C:\Users\zhuhe\AppData\Local\Temp\ipykernel_33196\3774832301.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_high_freq['RATIO'][i]= 1


In [4]:
data_high_freq_k = data_high_freq[["TOTAL_DSTR","VOICEREV_TOTAL","DATAREV_TOTAL","SMSREV_TOTAL",
                                 "RG_DAYS",'PRICE_PLAN','HS_CONN','HS_TYPE','VOL_KB','MO_MOU','RATIO']]
data_low_freq_k = data_low_freq[["TOTAL_DSTR","VOICEREV_TOTAL","DATAREV_TOTAL","SMSREV_TOTAL",
                                 "RG_DAYS",'PRICE_PLAN','HS_CONN','HS_TYPE', 'VOL_KB','MO_MOU','RATIO']]

In [5]:
data = pd.concat([data_low_freq_k,data_high_freq_k])

In [6]:
## change column names
c=['Total Revenue','Total Voice Revenue Total','Total Data Revenue','Total SMS Revenue','Active Days','Price Plan','Connection Type'
   ,'Handset Type', 'Charged Data Usage', 'Outgoing Minute Usage', 'Ratio']
data.columns = c
data

,Total Revenue,Total Voice Revenue Total,Total Data Revenue,Total SMS Revenue,Active Days,Price Plan,Connection Type,Handset Type,Charged Data Usage,Outgoing Minute Usage,Ratio
0,1.38,1.38,0.00,0.00,10,NISHCHINTO,4G,SMARTPHONE 4G,0.00,1.53,0.0
1,11.96,11.96,0.00,0.00,15,NISHCHINTO,BASIC,BASIC-VOICE ONLY,0.00,16.90,0.0
2,20.57,19.32,0.75,0.50,4,DJUICE_ADJUSTED,4G,SMARTPHONE 4G,769.14,9.53,0.0
3,58.59,54.59,0.00,0.00,43,NISHCHINTO,2G,BASIC-DATA CAPABLE,1.13,78.25,0.0
4,37.63,37.63,0.00,0.00,8,NISHCHINTO,2G,BASIC-DATA CAPABLE,1.12,22.10,0.0
...,...,...,...,...,...,...,...,...,...,...,...
81604,741.47,335.78,405.67,0.00,84,DJUICE_ADJUSTED,4G,SMARTPHONE 4G,29937059.73,486.08,2.0
81605,297.75,297.75,0.00,0.00,76,NISHCHINTO,BASIC,BASIC-VOICE ONLY,1.78,507.72,2.0
81606,288.92,129.27,158.66,1.00,69,PREPAID SMILE PSTN,3G,SMARTPHONE,8530344.97,177.48,2.0
81607,572.11,275.92,162.72,133.45,80,DJUICE_ADJUSTED,4G,SMARTPHONE 4G,8130163.68,338.43,2.0


In [7]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
data[['Price Plan','Connection Type','Handset Type']]=data[['Price Plan','Connection Type','Handset Type']].apply(le.fit_transform)

In [8]:
data.iloc[:,:-1]

,Total Revenue,Total Voice Revenue Total,Total Data Revenue,Total SMS Revenue,Active Days,Price Plan,Connection Type,Handset Type,Charged Data Usage,Outgoing Minute Usage
0,1.38,1.38,0.00,0.00,10,6,2,3,0.00,1.53
1,11.96,11.96,0.00,0.00,15,6,3,1,0.00,16.90
2,20.57,19.32,0.75,0.50,4,3,2,3,769.14,9.53
3,58.59,54.59,0.00,0.00,43,6,0,0,1.13,78.25
4,37.63,37.63,0.00,0.00,8,6,0,0,1.12,22.10
...,...,...,...,...,...,...,...,...,...,...
81604,741.47,335.78,405.67,0.00,84,3,2,3,29937059.73,486.08
81605,297.75,297.75,0.00,0.00,76,6,3,1,1.78,507.72
81606,288.92,129.27,158.66,1.00,69,10,1,2,8530344.97,177.48
81607,572.11,275.92,162.72,133.45,80,3,2,3,8130163.68,338.43


In [9]:
from sklearn.model_selection import train_test_split
source_X = data.iloc[:,:-1]
source_Y = data['Ratio']

#traning dataset and test dataset
train_X, test_X, train_y, test_y = train_test_split(source_X ,
                                                    source_Y,
                                                    train_size=.8)

In [10]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

#build the model
model.fit(train_X, train_y )

LogisticRegression()

In [11]:
model.score(test_X, test_y )

0.7239282303864905